In [ ]:
 ## GET helper functions
import sys
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import helper_functions
import tensorflow as tf
import tensorflow_hub as hub
SAVE_DIR = "model_logs"
# Function to evaluate : accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys, calculate_results

In [ ]:
helper_functions.download_file("https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip","nlp_getting_started.zip")
helper_functions.unzip_data("nlp_getting_started.zip")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Shuffle training dataframe
# frac : nb of % to shuffle 1=100%
train_df_shuffled = train_df.sample(frac=1, random_state = 42)
test_df_shuffled = test_df.sample(frac=1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
#  Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of  samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target:{target}", "(real diaster)" if target > 0 else "(not real diaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target:0 (not real diaster)
Text:
If I can't ruin his mood then I may have lost my direction. https://t.co/sLc27EMUgM

---

Target:0 (not real diaster)
Text:
I made a friend more interesting than all of y'all combined https://t.co/isBtzUJFBm

---

Target:0 (not real diaster)
Text:
her eyes and words are so icy but she burns like rum on the fire

---

Target:1 (real diaster)
Text:
On #ThisDayInHistory in 1862 Confederate ship blown up by crew. Read More http://t.co/IW7ELSzIfZ via @History

---

Target:0 (not real diaster)
Text:
@montetjwitter11 @nolesfan05 @NutsAndBoltsSP I say concern but not panic. Too many games left. And some games left against each other.

---



In [ ]:
from sklearn.model_selection import train_test_split

# turn dataframe values to dataframe columns
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                    train_df_shuffled["target"].to_numpy(),
                                                    test_size = 0.1, # use 10% of traianing data for validations
                                                    random_state = 42)
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

### Vectorizing and embedding

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be 

# Create a text vectorizer 
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                   output_mode = "int",
                                   output_sequence_length= max_length)

# Fit the text vectorizer to the training text to convert words to numeric format
text_vectorizer.adapt(train_sentences)

# Choose a random sentence from the training dataset and tokenize it 
random_sentence = random.choice(train_sentences)
text_vectorizer([random_sentence])

# Creating an Embedding using an Embedding Layer
# Use TensorFlow's embedding layer to create an embedding
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length)

# Embed the random sentence (turn it into vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
random_sentence, sample_embed

("@joonma_ dealbreaker that's it that's the dealbreaker s.o.s. abandon ship evacuate the building",
 <tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[-0.02840843,  0.03558198, -0.01696192, ...,  0.03411793,
          -0.00848315, -0.03471863],
         [-0.03166925, -0.03218926,  0.01545706, ...,  0.02897281,
          -0.04358655, -0.01922547],
         [-0.03228559,  0.04981809,  0.02190939, ...,  0.02613357,
           0.04990527, -0.04656596],
         ...,
         [ 0.00868673, -0.03709521,  0.01505773, ..., -0.0021978 ,
          -0.04726912,  0.03231165],
         [ 0.04823471,  0.04602   ,  0.00756792, ..., -0.01552367,
           0.01390021,  0.02851016],
         [ 0.04823471,  0.04602   ,  0.00756792, ..., -0.01552367,
           0.01390021,  0.02851016]]], dtype=float32)>)

## Question 1 : Rebuild, compile and train model_1, model_2 and model_5 using the Keras Sequential API instead of the Functional API.

### Model 1 simple dense model

In [ ]:
# # Model 1: A simple dense model

# Build model with Sequantial API
from tensorflow.keras import layers
model_1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype=tf.string),
    text_vectorizer,
    embedding,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1, activation="sigmoid"),
], name="dense_model")

model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

history_model_1 = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                        experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230128-081522
Epoch 1/5
215/215 [==============================] - 4s 5ms/step - loss: 0.6113 - accuracy: 0.7073 - val_loss: 0.5364 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4408 - accuracy: 0.8184 - val_loss: 0.4705 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3460 - accuracy: 0.8610 - val_loss: 0.4585 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2836 - accuracy: 0.8921 - val_loss: 0.4624 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2368 - accuracy: 0.9127 - val_loss: 0.4783 - val_accuracy: 0.7913


In [ ]:
model_1.summary()

Model: "dense_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

# Make some predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape, model_1_pred_probs[0]
# Convert model prediction probabilities to label format

model_1_pred_probs = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred_probs[:20]


# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                   y_pred=model_1_pred_probs)
model_1_results

24/24 [==============================] - 0s 2ms/step


{'accuracy': 79.13385826771653,
 'precision': 0.7949959084822513,
 'recall': 0.7913385826771654,
 'f1': 0.7888851650587367}

### Model 2 LSTM

In [ ]:

# ### Model 2 : LSTM 
# LSTM = long short term memory (one of the most popular LSTM cells)
# 
# Our structure of an RNN Typically looks like this:
# 
# ```
# Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
# ```

# Create an LSTM model
from tensorflow.keras import layers
model_2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype="string"),
    text_vectorizer,
    embedding,
    tf.keras.layers.LSTM(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")

], name="model_2_LSTM")

# Get the summary 
model_2.summary()


# Compile the model
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


# Fit the model 
history_model2 = model_2.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_2_LSTM")]) 

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________
Saving TensorBoard log files to: model_logs/model_2_LSTM/20230128-081530
Epoch 1/5
215/215 [==================

In [ ]:
# Make prediction with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs)) # enlever les dimension en plus
model_2_preds[:10]


# Calculate model 2 results
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

24/24 [==============================] - 0s 4ms/step


{'accuracy': 77.16535433070865,
 'precision': 0.7720646185150423,
 'recall': 0.7716535433070866,
 'f1': 0.7702930138828643}

### Model 5 Conv1D

In [ ]:

# ## Model 5 : Conv1D
# 
# For different explanations of parameters see:
# * https://poloclub.github.io/cnn-explainer/
# * Difference between "same" and "valid" padding:https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t#:~:text=To%20sum%20up%2C%20'valid',same'%20padding%20means%20using%20padding.



# embedding_test = embedding(text_vectorizer(["c est test de phrase"]))
# conv_1d = tf.keras.layers.Conv1D(filters=32,
#                                  kernel_size=5 , 
#                                  activation="relu",
#                                  padding="valid")
# conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
# max_pool = tf.keras.layers.GlobalAvgPool1D()
# max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the feature with the highest value"

# embedding_test.shape, conv_1d_output.shape, max_pool_output.shape
# # embedding_test
# # conv_1d_output
# # max_pool_output



# Test out our embedding layer, Conv1D layer and max pooling
model_5 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1), dtype=tf.string),
    text_vectorizer,
    embedding,
    tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5 , 
                           activation="relu",
                           strides=1,
                           padding="valid"),
    tf.keras.layers.GlobalAvgPool1D(),
    tf.keras.layers.Dense(1, activation="sigmoid")
], name="model_5_Conv1d")

model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
# Get a summary of our Conv1D model
model_5.summary()


model_5.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                 experiment_name="model_5_Conv1d")])



model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs
# convert model_5_pred_probs to labels 0 or 1
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
# model_5_preds


model_5_results = calculate_results(val_labels, model_5_preds)
model_5_results

Model: "model_5_Conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 64)            41024     
                                                                 
 global_average_pooling1d_1   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,321,089
Trainable params: 1,321,089


{'accuracy': 76.64041994750657,
 'precision': 0.7662606650453277,
 'recall': 0.7664041994750657,
 'f1': 0.7654276516145048}

## Question 2 : Retrain the baseline model with 10% of the training data. How does perform compared to the Universal Sentence Encoder model with 10% of the training data?

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline ([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text clf : juste classification
])

# Fit the pipleine to the training data
model_0.fit(train_sentences, train_labels)


# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score}")

# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

Our baseline model achives an accuracy of : 0.7926509186351706


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [ ]:
# Train model with 10% of data
len_train_10_percent = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:len_train_10_percent]
train_labels_10_percent = val_labels[:len_train_10_percent]
train_sentences_10_percent.shape, train_labels_10_percent.shape

# Fit the pipleine to the training data
model_0.fit(train_sentences_10_percent, train_labels_10_percent)

# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score}")

# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

Our baseline model achives an accuracy of : 0.515748031496063


{'accuracy': 51.574803149606296,
 'precision': 0.49141412143034424,
 'recall': 0.515748031496063,
 'f1': 0.4732025629390422}

In [ ]:
model_7_results = {'accuracy': 78.4776902887139,
                   'precision': 0.7870511640590381,
                   'recall': 0.7847769028871391,
                   'f1': 0.7827022002767159}

In [ ]:
import numpy as pd
np.array(list(model_7_results.values())) > pd.array(list(baseline_results.values()))
# Conclusion : model 7 Universal Sentence Encoder is the best 

array([ True,  True,  True,  True])

## Question 3 : Try fine-tuning the TF Hub Universal Sentence Encoder model by setting training=True when instantiating it as a Keras layer.

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=True,
                                        name="USE")

In [ ]:
# Create model using the Sequential API
model_6_v1 = tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid", name="output_layer")
], name="model_6_USE")

model_6_v1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Train a classifier on top of USE pretrained embeddings
history_model_6_v1 = model_6_v1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                  experiment_name="model_6_tf_hub")])


Saving TensorBoard log files to: model_logs/model_6_tf_hub/20230128-081635
Epoch 1/5
215/215 [==============================] - 39s 154ms/step - loss: 0.4612 - accuracy: 0.7889 - val_loss: 0.4112 - val_accuracy: 0.8255
Epoch 2/5
215/215 [==============================] - 32s 151ms/step - loss: 0.2417 - accuracy: 0.9050 - val_loss: 0.4712 - val_accuracy: 0.8255
Epoch 3/5
215/215 [==============================] - 33s 153ms/step - loss: 0.0751 - accuracy: 0.9764 - val_loss: 0.6687 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 33s 151ms/step - loss: 0.0410 - accuracy: 0.9850 - val_loss: 0.6181 - val_accuracy: 0.8031
Epoch 5/5
215/215 [==============================] - 33s 151ms/step - loss: 0.0271 - accuracy: 0.9882 - val_loss: 0.6719 - val_accuracy: 0.7966


In [ ]:
# Make prediction with USE TF Hub Model
model_6_v1_pred_probs = model_6_v1.predict(val_sentences)
model_6_v1_preds = tf.squeeze(tf.round(model_6_v1_pred_probs)) # transforme to labels

24/24 [==============================] - 0s 9ms/step


In [ ]:
# Calculate model_6 performance metrics
model_6_results = calculate_results(val_labels,model_6_v1_preds)
model_6_results

{'accuracy': 79.65879265091863,
 'precision': 0.796388851341711,
 'recall': 0.7965879265091863,
 'f1': 0.7960876878433872}

## Question 4 : Retrain the best model you've got so far on the whole training set (no validation split). Then use this trained model to make predictions on the test dataset and format the predictions into the same format as the sample_submission.csv file from Kaggle (see the Files tab in Colab for what the sample_submission.csv file looks like). Once you've done this, [make a submission to the Kaggle](https://www.kaggle.com/c/nlp-getting-started/data) competition, how did your model perform?

In [ ]:
test_df_shuffled

,id,keyword,location,text
2406,8051,refugees,NaN,Refugees as citizens - The Hindu http://t.co/G...
134,425,apocalypse,Currently Somewhere On Earth,@5SOStag honestly he could say an apocalypse i...
411,1330,blown%20up,Scout Team,If you bored as shit don't nobody fuck wit you...
203,663,attack,NaN,@RealTwanBrown Yesterday I Had A Heat Attack ?...
889,2930,danger,Leeds,The Devil Wears Prada is still one of my favou...
...,...,...,...,...
1095,3606,desolation,NaN,Free Kindle Book - Aug 3-7 - Thriller - Desola...
1130,3731,destroyed,Virginia,HitchBot travels Europe and greeted with open ...
1294,4259,drowning,Doo Doo Boy Island,If you told me you was drowning. I would not l...
860,2816,cyclone,"Brooklyn, NY",First time for everything! @ Coney Island Cycl...


In [ ]:
# from sklearn.model_selection import train_test_split

# # turn dataframe values to dataframe columns
# train_sentences_s, train_labels_s, _,_ = train_test_split(train_df_shuffled["text"].to_numpy(),
#                                                     train_df_shuffled["target"].to_numpy(),
#                                                     test_size = 1, # use 10% of traianing data for validations
#                                                     random_state = 42)
# # Check the lengths
# len(train_sentences_s), len(train_labels_s)

In [ ]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid", name="output_layer")
], name="model_6_USE")

model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Train a classifier on top of USE pretrained embeddings
history_model_6 = model_6.fit(train_df_shuffled["text"].to_numpy(),
                              train_df_shuffled["target"].to_numpy(),
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                  experiment_name="model_6_tf_hub")])

Saving TensorBoard log files to: model_logs/model_6_tf_hub/20230128-082003
Epoch 1/5
238/238 [==============================] - 40s 152ms/step - loss: 0.1184 - accuracy: 0.9678 - val_loss: 0.2453 - val_accuracy: 0.9121
Epoch 2/5
238/238 [==============================] - 36s 150ms/step - loss: 0.0452 - accuracy: 0.9849 - val_loss: 0.0782 - val_accuracy: 0.9751
Epoch 3/5
238/238 [==============================] - 36s 151ms/step - loss: 0.0224 - accuracy: 0.9912 - val_loss: 0.0513 - val_accuracy: 0.9816
Epoch 4/5
238/238 [==============================] - 36s 150ms/step - loss: 0.0172 - accuracy: 0.9924 - val_loss: 0.0192 - val_accuracy: 0.9895
Epoch 5/5
238/238 [==============================] - 36s 150ms/step - loss: 0.0155 - accuracy: 0.9928 - val_loss: 0.0148 - val_accuracy: 0.9974


In [ ]:
model_6_pred_probs = model_6.predict(test_df_shuffled["text"])
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

102/102 [==============================] - 1s 8ms/step


In [ ]:
import pandas as pd
# Convert to dataframe : facilité 
df_model_6_preds = pd.DataFrame(model_6_preds)

df_model_6_preds_id_target = pd.DataFrame({'id': test_df_shuffled["id"],
                                           'target': df_model_6_preds[0].astype(int)})

df_model_6_preds_id_target = df_model_6_preds_id_target.sort_values(['target'], ascending=[False])
df_model_6_preds_id_target

df_model_6_preds_id_target.to_csv('nlp_disaster_tweets_submission.csv', index=False)

In [ ]:
df_model_6_preds_id_target

,id,target
0,0,1
1838,6210,1
1831,6186,1
1832,6189,1
1833,6190,1
...,...,...
1897,6401,0
1896,6390,0
1895,6381,0
697,2268,0


In [ ]:
test_df_shuffled.index

Int64Index([2406,  134,  411,  203,  889, 1432, 3024, 2741,  463,  291,
            ...
            1238,  466, 2169, 1638, 3092, 1095, 1130, 1294,  860, 3174],
           dtype='int64', length=3263)

In [ ]:
# for i, row in enumerate(df_model_6_preds.itertuples(), 1):
#   print(f"id : {row.Index}, Target :{row._1}")
import pandas as pd

# Table de prédiction en utilisant un DataFrame pandas
prediction_table = pd.DataFrame({
    'id': [1, 2, 3],
    'target': ['positive', 'negative', 'neutral']
})

# Enregistrement du DataFrame dans un fichier CSV


In [ ]:
# regarde le format des fichier demander pour adapté le resultat de predict s'il faut

## Question 5 : Combine the ensemble predictions using the majority vote (mode), how does this perform compare to averaging the prediction probabilities of each model?

## Question 6 : Make a confusion matrix with the best performing model's predictions on the validation set and the validation ground truth labels.

In [ ]:
from helper_functions import confusion_matrix
confusion_matrix(y_true=val_labels,
                 y_pred=model_6_v1_preds)

array([[345,  69],
       [ 86, 262]])

In [ ]:
# Ici vous avez terminer toute les etapes a part la 5eme question

In [ ]:
import pandas as pd

# Load the submission file into a Pandas DataFrame
submission = pd.read_csv("prediction_example.csv")

# Load the list of required key values
with open("required_skeys.txt", "r") as f:
    required_keys = f.read().splitlines()

# Check if all the required key values are present in the 'id' column
missing_keys = set(required_keys) - set(submission["id"])
if len(missing_keys) > 0:
    print(f"Missing key values: {missing_keys}")
else:
    print("All required key values are present in the 'id' column.")